In [5]:
from ultralytics import YOLO

import cv2
import matplotlib.pyplot as plt


In [2]:
model = YOLO("yolo11n.pt")


In [6]:
def visualize_results(results): 
    for result in results:
        img = result.orig_img  # Original image used for inference
    
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Top-left and bottom-right corner coordinates
            label = result.names[int(box.cls[0])]  # Class label
            confidence = box.conf[0]  # Confidence score
            
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue box
            cv2.putText(img, f"{label} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()

In [4]:
predictions = model.predict(source="./datasets/DAWN/Fog/foggy-011.jpg", show=True)

2024-11-22 17:30:31.792 Python[60288:14105010] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-22 17:30:31.792 Python[60288:14105010] +[IMKInputSession subclass]: chose IMKInputSession_Modern


image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/DAWN/Fog/foggy-011.jpg: 384x640 1 car, 7 traffic lights, 39.9ms
Speed: 1.6ms preprocess, 39.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)


In [8]:
import os
import numpy as np


def load_ground_truth(file_path, img_width, img_height):
    boxes = []
    with open(file_path, 'r') as file:
        for line in file:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            x1 = int((x_center - width / 2) * img_width)
            y1 = int((y_center - height / 2) * img_height)
            x2 = int((x_center + width / 2) * img_width)
            y2 = int((y_center + height / 2) * img_height)
            boxes.append((int(class_id), x1, y1, x2, y2))
    return boxes

def calculate_iou(boxA, boxB):
    xA = max(boxA[1], boxB[1])
    yA = max(boxA[2], boxB[2])
    xB = min(boxA[3], boxB[3])
    yB = min(boxA[4], boxB[4])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    
    boxAArea = (boxA[3] - boxA[1]) * (boxA[4] - boxA[2])
    boxBArea = (boxB[3] - boxB[1]) * (boxB[4] - boxB[2])
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def evaluate_model_on_folder(folder_path, gt_folder, model):
    all_ious = []
    for img_name in os.listdir(folder_path):
        if not img_name.endswith(".jpg"):
            continue
        img_path = os.path.join(folder_path, img_name)
        gt_path = os.path.join(gt_folder, os.path.splitext(img_name)[0] + ".txt")
        
        img = cv2.imread(img_path)
        img_height, img_width = img.shape[:2]
        
        gt_boxes = load_ground_truth(gt_path, img_width, img_height)
        results = model.predict(source=img_path, show=False)
        
        for pred in results[0].boxes:
            pred_class = int(pred.cls[0])
            pred_box = [pred_class] + list(map(int, pred.xyxy[0].tolist()))
            best_iou = 0.0
            for gt_box in gt_boxes:
                if pred_class == gt_box[0]:
                    iou = calculate_iou(pred_box, gt_box)
                    best_iou = max(best_iou, iou)
            all_ious.append(best_iou)
    
    mean_iou = np.mean(all_ious)
    return mean_iou



In [21]:
def txt_move_to(root_folder, dest_folder):
    os.makedirs(dest_folder, exist_ok=True)

    for file_name in os.listdir(root_folder):
        if file_name.endswith(".txt"):  # Check if the file ends with .txt
            src_path = os.path.join(root_folder, file_name)
            dst_path = os.path.join(dest_folder, file_name)
            shutil.move(src_path, dst_path)  # Move the file
            print(f"Moved: {file_name} -> {dest_folder}")

In [27]:
txt_move_to("./datasets/DAWN_split/train/Fog", "./datasets/DAWN_split/train/Fog_gt")

Moved: haze-080.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: foggy-030.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-094.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-043.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-057.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-122.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-136.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-042.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-095.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: foggy-031.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-081.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: foggy-019.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-097.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-083.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-068.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-054.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: haze-040.txt -> ./datasets/DAWN_split/train/Fog_gt
Moved: mist

In [28]:
txt_move_to("./datasets/DAWN_split/train/Rain", "./datasets/DAWN_split/train/Rain_gt")
txt_move_to("./datasets/DAWN_split/train/Sand", "./datasets/DAWN_split/train/Sand_gt")
txt_move_to("./datasets/DAWN_split/train/Snow", "./datasets/DAWN_split/train/Snow_gt")


Moved: rain_storm-019.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-757.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-233.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-435.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-186.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-839.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-811.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-150.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-385.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-597.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-030.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-018.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-740.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-026.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-783.txt -> ./datasets/DAWN_split/train/Rain_gt
Moved: rain_storm-146.txt

In [30]:
txt_move_to("./datasets/DAWN_split/val/Fog", "./datasets/DAWN_split/val/Fog_gt")
txt_move_to("./datasets/DAWN_split/val/Rain", "./datasets/DAWN_split/val/Rain_gt")
txt_move_to("./datasets/DAWN_split/val/Sand", "./datasets/DAWN_split/val/Sand_gt")
txt_move_to("./datasets/DAWN_split/val/Snow", "./datasets/DAWN_split/val/Snow_gt")

Moved: foggy-018.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: foggy-024.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: mist-013.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: haze-056.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: foggy-025.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: foggy-033.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: foggy-027.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: haze-135.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: haze-041.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: mist-955.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: haze-050.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: haze-087.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: haze-093.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: haze-046.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: foggy-034.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: foggy-047.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: haze-020.txt -> ./datasets/DAWN_split/val/Fog_gt
Moved: mist-064.txt -> ./datasets/DAWN_sp

In [17]:
# Run evaluation on each folder (e.g., Fog, Rain, etc.)
evaluate_model_on_folder("./datasets/DAWN/Fog", "./datasets/DAWN/Fog/Fog_YOLO_darknet", model)


image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/DAWN/Fog/mist-064.jpg: 448x640 3 cars, 1 truck, 78.2ms
Speed: 4.2ms preprocess, 78.2ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/DAWN/Fog/haze-141.jpg: 384x640 1 car, 1 bus, 1 truck, 48.9ms
Speed: 1.6ms preprocess, 48.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/DAWN/Fog/foggy-046.jpg: 448x640 1 car, 48.9ms
Speed: 2.3ms preprocess, 48.9ms inference, 0.7ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/DAWN/Fog/foggy-091.jpg: 416x640 1 car, 2 traffic lights, 48.3ms
Speed: 1.8ms preprocess, 48.3ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/DAWN/Fog/foggy-085.jpg: 41

0.6710752492047246

In [15]:
class_names = model.names  # Dictionary {index: 'classname'}
print("Class Indices and Names:")
for idx, name in class_names.items():
    print(f"Index: {idx}, Name: {name}")

Class Indices and Names:
Index: 0, Name: person
Index: 1, Name: bicycle
Index: 2, Name: car
Index: 3, Name: motorcycle
Index: 4, Name: airplane
Index: 5, Name: bus
Index: 6, Name: train
Index: 7, Name: truck
Index: 8, Name: boat
Index: 9, Name: traffic light
Index: 10, Name: fire hydrant
Index: 11, Name: stop sign
Index: 12, Name: parking meter
Index: 13, Name: bench
Index: 14, Name: bird
Index: 15, Name: cat
Index: 16, Name: dog
Index: 17, Name: horse
Index: 18, Name: sheep
Index: 19, Name: cow
Index: 20, Name: elephant
Index: 21, Name: bear
Index: 22, Name: zebra
Index: 23, Name: giraffe
Index: 24, Name: backpack
Index: 25, Name: umbrella
Index: 26, Name: handbag
Index: 27, Name: tie
Index: 28, Name: suitcase
Index: 29, Name: frisbee
Index: 30, Name: skis
Index: 31, Name: snowboard
Index: 32, Name: sports ball
Index: 33, Name: kite
Index: 34, Name: baseball bat
Index: 35, Name: baseball glove
Index: 36, Name: skateboard
Index: 37, Name: surfboard
Index: 38, Name: tennis racket
Index:

In [9]:
import shutil
import random

In [26]:
# dataset_dir = "./datasets/DAWN"
# categories = ["Fog", "Rain", "Snow", "Sand"]
#
# # Directories to store training and validation sets
#
# train_dir = "./datasets/DAWN_split/train"
# val_dir = "./datasets/DAWN_split/val"
# # Create directories if they don’t exist
# for folder in [train_dir, val_dir]:
#     os.makedirs(folder, exist_ok=True)
#     for cat in categories:
#         os.makedirs(os.path.join(folder, cat), exist_ok=True)
#
# # Split data into train and validation sets
# def split_data(train_ratio=0.8):
#     for category in categories:
#         category_dir = os.path.join(dataset_dir, category)
#         images = [f for f in os.listdir(category_dir) if f.endswith('.jpg')]
#         random.shuffle(images)
#
#         split_idx = int(len(images) * train_ratio)
#         train_images = images[:split_idx]
#         val_images = images[split_idx:]
#
#         for img_name in train_images:
#             shutil.copy(os.path.join(category_dir, img_name), os.path.join(train_dir, category, img_name))
#             txt_name = img_name.replace(".jpg", ".txt")
#             if os.path.exists(os.path.join(category_dir, f"{category}_YOLO_darknet", txt_name)):
#                 shutil.copy(
#                     os.path.join(category_dir, f"{category}_YOLO_darknet", txt_name),
#                     os.path.join(train_dir, category, txt_name)
#                 )
#
#         for img_name in val_images:
#             shutil.copy(os.path.join(category_dir, img_name), os.path.join(val_dir, category, img_name))
#             txt_name = img_name.replace(".jpg", ".txt")
#             if os.path.exists(os.path.join(category_dir, f"{category}_YOLO_darknet", txt_name)):
#                 shutil.copy(
#                     os.path.join(category_dir, f"{category}_YOLO_darknet", txt_name),
#                     os.path.join(val_dir, category, txt_name)
#                 )
#
# # Split data
# split_data()

In [9]:
model = YOLO("yolo11n.pt")

# Evaluate before training

In [31]:
train_dir = "./datasets/DAWN_split/train"
val_dir = "./datasets/DAWN_split/val"

In [35]:
print("Evaluating before training...")
initial_iou = evaluate_model_on_folder("./datasets/Dawn.v1i.yolov11/valid/images", "./datasets/Dawn.v1i.yolov11/valid/labels", model)
print("Initial Mean IoU on validation set:", initial_iou)

Evaluating before training...

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/haze-013_jpg.rf.0e7fec18de946008e2e0ceee60cc45e7.jpg: 640x640 (no detections), 113.0ms
Speed: 30.2ms preprocess, 113.0ms inference, 6.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/mist-098_jpg.rf.9f0bab09b56e4b13d6cf6e6c72cd88a2.jpg: 640x640 1 car, 61.6ms
Speed: 2.1ms preprocess, 61.6ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/sand_storm-281_jpg.rf.434f3007f121b710dde7af89981161e3.jpg: 640x640 6 cars, 1 truck, 58.3ms
Speed: 1.5ms preprocess, 58.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/rain_storm-265

In [36]:
for k, v in model.named_parameters():
    print(k)

model.model.0.conv.weight
model.model.0.conv.bias
model.model.1.conv.weight
model.model.1.conv.bias
model.model.2.cv1.conv.weight
model.model.2.cv1.conv.bias
model.model.2.cv2.conv.weight
model.model.2.cv2.conv.bias
model.model.2.m.0.cv1.conv.weight
model.model.2.m.0.cv1.conv.bias
model.model.2.m.0.cv2.conv.weight
model.model.2.m.0.cv2.conv.bias
model.model.3.conv.weight
model.model.3.conv.bias
model.model.4.cv1.conv.weight
model.model.4.cv1.conv.bias
model.model.4.cv2.conv.weight
model.model.4.cv2.conv.bias
model.model.4.m.0.cv1.conv.weight
model.model.4.m.0.cv1.conv.bias
model.model.4.m.0.cv2.conv.weight
model.model.4.m.0.cv2.conv.bias
model.model.5.conv.weight
model.model.5.conv.bias
model.model.6.cv1.conv.weight
model.model.6.cv1.conv.bias
model.model.6.cv2.conv.weight
model.model.6.cv2.conv.bias
model.model.6.m.0.cv1.conv.weight
model.model.6.m.0.cv1.conv.bias
model.model.6.m.0.cv2.conv.weight
model.model.6.m.0.cv2.conv.bias
model.model.6.m.0.cv3.conv.weight
model.model.6.m.0.cv3.

In [49]:
# Train the model on the split data
model = YOLO("yolo11n.pt")
print("Training model...")
results = model.train(data="/Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/data.yaml", epochs=50, imgsz=400, device="mps", pretrained=True, lr0=1e-4, plots=True)  # Adjust epochs as needed


Training model...
New https://pypi.org/project/ultralytics/8.3.36 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.28 🚀 Python-3.12.4 torch-2.5.1 MPS (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=400, save=True, save_period=-1, cache=False, device=mps, workers=8, project=None, name=train13, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classe

train: Scanning /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/train/labels... 2099 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2099/2099 [00:00<00:00, 3477.04it/s]

train: New cache created: /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/train/labels.cache



val: Scanning /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/labels... 197 images, 0 backgrounds, 0 corrupt: 100%|██████████| 197/197 [00:00<00:00, 3257.50it/s]

val: New cache created: /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/labels.cache


...

WARNING ⚠️ NMS time limit 2.250s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [01:29<00:00, 12.73s/it]


                   all        197       1471      0.441     0.0649      0.069     0.0388

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      12/50      6.62G      1.359       1.04      1.125        130        416:  77%|███████▋  | 102/132 [05:07<01:30,  3.02s/it]


RuntimeError: shape mismatch: value tensor of shape [13571] cannot be broadcast to indexing result of shape [23994]

I ran out of memory here(

In [12]:
import torch


In [51]:
torch.save(model.state_dict(), 'yolov11x_model_state.pt')

In [15]:
model = YOLO("/Users/vahe/Downloads/DL_homeworks/semester_project/runs/detect/train13/weights/best.pt")

In [16]:
# Evaluate after training
print("Evaluating after training...")
final_iou = evaluate_model_on_folder("./datasets/Dawn.v1i.yolov11/valid/images", "./datasets/Dawn.v1i.yolov11/valid/labels", model)
print("Final Mean IoU on validation set:", final_iou)

Evaluating after training...

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/haze-013_jpg.rf.0e7fec18de946008e2e0ceee60cc45e7.jpg: 416x416 1 0, 3 2s, 47.0ms
Speed: 5.7ms preprocess, 47.0ms inference, 4.7ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/mist-098_jpg.rf.9f0bab09b56e4b13d6cf6e6c72cd88a2.jpg: 416x416 2 2s, 35.1ms
Speed: 1.1ms preprocess, 35.1ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/sand_storm-281_jpg.rf.434f3007f121b710dde7af89981161e3.jpg: 416x416 6 2s, 34.5ms
Speed: 0.8ms preprocess, 34.5ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 416)

image 1/1 /Users/vahe/Downloads/DL_homeworks/semester_project/datasets/Dawn.v1i.yolov11/valid/images/rain_storm-265_jpg.rf.5fde1cbfde0731